# RNN

In [1]:
# 导入
import sys

sys.path.append("E:/dataFiles/github/MFlow")

In [2]:
# 数据生成
from data.generater import waveData

seq_len = 96  # 序列长度
in_dim = 16  # 输入维度
state_dim = 12  # 状态维度

xs, ys = waveData(1000, in_dim, seq_len)

train_xs = xs[:700]
train_ys = ys[:700]
test_xs = xs[700:]
test_ys = ys[700:]

print(train_xs.shape, train_ys.shape)
print(train_xs[0], train_ys[0])

(700, 96, 16) (700, 2)
[[-0.19651586 -0.10735795 -0.57027595 ...  0.81843166 -0.65288486
   0.13831083]
 [ 0.95872117  0.44811327  0.22441193 ...  0.65407623  0.20270836
  -0.36168748]
 [ 0.01120725 -0.29253854  0.35298953 ... -1.00774508  0.8626005
   0.63204357]
 ...
 [-0.82983572 -0.33686877 -0.13705201 ... -0.74389747  0.31126875
  -0.80513672]
 [-0.75449754 -0.37745461 -0.3922741  ... -0.60842135 -1.17763147
  -0.3696263 ]
 [ 0.07101136 -1.21775053 -0.80890495 ...  0.54379608 -0.3019266
  -0.54774718]] [1. 0.]


In [3]:
# 训练
import numpy as np
from mflow import core, ops, opts, lays

# 超参数
lr = 0.005
epoch = 30
batch_size = 16

with core.NameScope("RNN"):
    # 初始化变量
    inputs = [core.Variable(size=(in_dim, 1), trainable=False) for _ in range(seq_len)]
    u = core.Variable(size=(state_dim, in_dim), trainable=True)
    w = core.Variable(size=(state_dim, state_dim), trainable=True)
    b = core.Variable(size=(state_dim, 1), trainable=True)
    y = core.Variable(size=(2, 1), trainable=False)
    last_step = None  # 上一步输出，第一步的话就设置为None
    # 网络构建
    for iv in inputs:
        h = ops.Add(ops.MatMul(u, iv), b)
        if last_step is not None:
            h = ops.Add(ops.MatMul(w, last_step), h)
        h = ops.ReLU(h)
        last_step = h
    fc_1 = lays.Linear(last_step, state_dim, 40, "ReLU")
    fc_2 = lays.Linear(fc_1, 40, 10, "ReLU")
    pred = lays.Linear(fc_2, 10, 2, None)
    predicter = ops.Logistic(pred)
    loss = ops.loss.CrossEntropyWithSoftMax(pred, y)
    loss.eps = 1e-8  # 避免梯度消失
    adam = opts.Adam(core.DefaultGraph, loss, lr)
    # 开始训练
    for ep in range(1, epoch + 1):
        bs_idx = 0  # 批次计数
        # 这是一个epoch的过程
        for i, (feat, lab) in enumerate(zip(train_xs, train_ys)):
            for j, x in enumerate(inputs):
                x.setValue(np.mat(feat[j]).T)
            y.setValue(np.mat(lab).T)
            adam.step()
            bs_idx += 1
            if bs_idx == batch_size:
                if (i + 1) % 64 == 0:
                    print("Epoch: {:d}, itet: {:d}, loss: {:.7f}.".format(
                        ep, i + 1, loss.value[0, 0]))
                adam.update()
                bs_idx = 0
        # 一个epoch完成后进行评估
        preds = []
        for feat in test_xs:
            for j, x in enumerate(inputs):
                x.setValue(np.mat(feat[j]).T)
            predicter.forward()
            preds.append(predicter.value.A.ravel())  # 结果
        preds = np.array(preds).argmax(axis=1)
        trues = test_ys.argmax(axis=1)
        acc = (trues == preds).astype("uint8").sum() / len(test_xs)
        print("Epoch: {:d}, acc: {:.3f}.".format(ep, acc))

Epoch: 1, itet: 64, loss: 0.7011864.
Epoch: 1, itet: 128, loss: 0.7243448.
Epoch: 1, itet: 192, loss: 0.6664761.
Epoch: 1, itet: 256, loss: 0.6662409.
Epoch: 1, itet: 320, loss: 0.7237994.
Epoch: 1, itet: 384, loss: 0.7207229.
Epoch: 1, itet: 448, loss: 0.6667717.
Epoch: 1, itet: 512, loss: 0.6709399.
Epoch: 1, itet: 576, loss: 0.6785114.
Epoch: 1, itet: 640, loss: 0.6963072.
Epoch: 1, acc: 0.477.
Epoch: 2, itet: 64, loss: 0.6653894.
Epoch: 2, itet: 128, loss: 0.6694875.
Epoch: 2, itet: 192, loss: 0.6234589.
Epoch: 2, itet: 256, loss: 0.1756821.
Epoch: 2, itet: 320, loss: 0.4252226.
Epoch: 2, itet: 384, loss: 0.6880452.
Epoch: 2, itet: 448, loss: 0.3213921.
Epoch: 2, itet: 512, loss: 0.4237661.
Epoch: 2, itet: 576, loss: 0.3314456.
Epoch: 2, itet: 640, loss: 0.5297717.
Epoch: 2, acc: 0.947.
Epoch: 3, itet: 64, loss: 0.4432591.
Epoch: 3, itet: 128, loss: 0.4441963.
Epoch: 3, itet: 192, loss: 0.1682618.
Epoch: 3, itet: 256, loss: 0.0350314.
Epoch: 3, itet: 320, loss: 0.3645093.
Epoch: 3,